In [1]:
import requests, re, time, logging
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def scrape_freelancer_jobs(max_jobs: int = 350,throttle: float = 1.0,verbose: bool = True) -> pd.DataFrame:
   
    FX = {"USD": 1.0, "INR": 0.012, "EUR": 1.08, "GBP": 1.27,
          "$": 1.0, "₹": 0.012, "€": 1.08, "£": 1.27}
    def _budget_to_usd(text: str) -> float:
        """Extract upper numeric bound and convert to USD."""
        nums = re.findall(r"[\d,]+(?:\.\d+)?", text.replace(",", ""))
        if not nums:
            return 0.0
        hi = float(nums[-1])
        cur = re.search(r"[$₹€£]|USD|INR|EUR|GBP", text.upper())
        rate = FX.get(cur.group(0), 1.0) if cur else 1.0
        return round(hi * rate, 2)
    headers = {"User-Agent": "Mozilla/5.0"}
    jobs, page = [], 1
    while len(jobs) < max_jobs:
        url = f"https://www.freelancer.com/jobs/{page}"
        r = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(r.text, "html.parser")
        cards = soup.select("div.JobSearchCard-item")
        if verbose:
            print(f"[page {page}] cards found: {len(cards)} "
                  f"(total collected: {len(jobs)})")
        if not cards:
            logging.warning("No cards returned – stopping early.")
            break
        for div in cards:
            if len(jobs) >= max_jobs:
                break 
            try:
                title_tag = div.select_one("a.JobSearchCard-primary-heading-link")
                title = title_tag.text.strip()
                link = "https://www.freelancer.com" + title_tag["href"]
                desc = div.select_one("p.JobSearchCard-primary-description").text.strip()
                skills = [s.text for s in div.select("a.JobSearchCard-primary-tagsLink")]
                price_raw = div.select_one("div.JobSearchCard-secondary-price").text.strip()
                budget_val = _budget_to_usd(price_raw)
                job_type = "Hourly" if "hour" in price_raw.lower() else "Fixed"
                jobs.append({
                    "platform": "Freelancer.com",
                    "job_title": title,
                    "category": "Mixed",
                    "skills_required": ", ".join(skills),
                    "job_type": job_type,
                    "budget_usd": budget_val,
                    "location_preference": "Worldwide",
                    "duration_estimate": "Short-term",
                    "url": link
                })
            except Exception as exc:
                logging.error(f"Parse error on page {page}: {exc}")
                continue
        page += 1
        time.sleep(throttle)
    df = pd.DataFrame(jobs[:max_jobs])  
    if verbose:
        print(f"Finished with {len(df)} rows")
    return df

In [3]:
df_freelancer = scrape_freelancer_jobs(max_jobs=350)
df_freelancer.to_csv("scraped_data.csv", index=False)

ERROR:root:❌ parse error on page 1: 'NoneType' object has no attribute 'text'
ERROR:root:❌ parse error on page 1: 'NoneType' object has no attribute 'text'


[page 1] cards found: 50 (total collected: 0)


ERROR:root:❌ parse error on page 2: 'NoneType' object has no attribute 'text'
ERROR:root:❌ parse error on page 2: 'NoneType' object has no attribute 'text'


[page 2] cards found: 50 (total collected: 48)
[page 3] cards found: 50 (total collected: 96)


ERROR:root:❌ parse error on page 4: 'NoneType' object has no attribute 'text'
ERROR:root:❌ parse error on page 4: 'NoneType' object has no attribute 'text'


[page 4] cards found: 50 (total collected: 146)
[page 5] cards found: 50 (total collected: 194)


ERROR:root:❌ parse error on page 6: 'NoneType' object has no attribute 'text'
ERROR:root:❌ parse error on page 6: 'NoneType' object has no attribute 'text'


[page 6] cards found: 50 (total collected: 244)


ERROR:root:❌ parse error on page 7: 'NoneType' object has no attribute 'text'
ERROR:root:❌ parse error on page 7: 'NoneType' object has no attribute 'text'


[page 7] cards found: 50 (total collected: 292)
[page 8] cards found: 50 (total collected: 340)
✅ finished with 350 rows
